In [48]:
import pandas as pd

C = pd.read_stata(
    "/Users/clarissaache/Documents/IDS 701/uds-2022-ids-701-team-3/00_source_data/0405_sec_c0.dta"
)
print(C.shape)

(91319, 21)


In [49]:
C.columns

Index(['hhcode', 'sec', 'msno', 'scq01', 'scq02', 'scq03', 'scqo4', 'scq05',
       'scq06', 'scq07', 'scq08a', 'scq08b', 'scq09a', 'scq09b', 'psu',
       'quarter', 'region', 'psu_hies', 'hhno', 'hhcode_hies', 'province'],
      dtype='object')

In [50]:
# Reduce to the questions of interest (4 questions):

C["ever_admitted"] = C["scq03"]
C["max_level_acheived"] = C["scqo4"]
C["currently_enrrolled"] = C["scq05"] # IMPORTANT!!! (counts of enrrollment)
C["why_not"] = C["scq09a"]

C_relevant = C[
    [
        "hhcode",
        "msno",
        "ever_admitted",
        "max_level_acheived",
        "currently_enrrolled",
        "why_not",
        "region",
        "province"
    ]
].copy()


In [51]:
C_relevant

,hhcode,msno,ever_admitted,max_level_acheived,currently_enrrolled,why_not,region,province
0,1.001100e+09,1,yes,class 10,no,2 education completed,urban,punjab
1,1.001100e+09,2,yes,cl;ass 8,no,2 education completed,urban,punjab
2,1.001100e+09,3,yes,17: ma/msc,no,2 education completed,urban,punjab
3,1.001100e+09,4,yes,class 10,yes,NaN,urban,punjab
4,1.001100e+09,1,yes,12: ba/bsc,no,10 employment,urban,punjab
...,...,...,...,...,...,...,...,...
91314,4.262002e+09,6,no,NaN,NaN,15 other,rural,balochistan
91315,4.262002e+09,1,no,NaN,NaN,10 employment,rural,balochistan
91316,4.262002e+09,2,no,NaN,NaN,13 parent did not approve,rural,balochistan
91317,4.262002e+09,3,no,NaN,NaN,13 parent did not approve,rural,balochistan


In [52]:
B = pd.read_stata(
    "/Users/clarissaache/Documents/IDS 701/uds-2022-ids-701-team-3/00_source_data/0405_sec_b0.dta"
)
B.sample(5)

# Change the names of the questions for readability
B['sex'] = B['sbq01']
B['age'] = B['sbq04']
B['marital_status'] = B['sbq05']

B_relevant = B[['hhcode', 'msno', 'age', 'sex', 'marital_status']].copy()

In [53]:
B.sample(5)

,hhcode,sec,msno,sbq01,sbq02,sbq03,sbq04,sbq05,weight,psu,quarter,region,psu_hies,hhno,hhcode_hies,province,sex,age,marital_status
83478,3.221001e+09,B,1,female,present,1:self,45,4:widow / widowers,107.856003,32210006,3,urban,30610302,9,3.061030e+09,nwfp,female,45,4:widow / widowers
2214,1.021000e+09,B,10,female,present,7: son in law / daughter in law,24,2:currently married,1219.088013,10210004,4,urban,10110401,5,1.011040e+09,punjab,female,24,2:currently married
4948,1.051001e+09,B,2,female,present,2: husband/wife,30,2:currently married,1579.657959,10510012,1,urban,10210102,2,1.021010e+09,punjab,female,30,2:currently married
18390,1.152002e+09,B,1,male,present,1:self,40,2:currently married,686.461975,11520018,3,rural,11520302,11,1.152030e+09,punjab,male,40,2:currently married
93149,4.102002e+09,B,3,male,present,3:son/daughter,20,1:never married,356.868988,41020017,4,rural,40320402,11,4.032040e+09,balochistan,male,20,1:never married


In [54]:
# Made the Household Code column a str type
B["hhcode"] = B["hhcode"].astype("str")
C["hhcode"] = C["hhcode"].astype("str")

In [55]:
# MERGE
years04_05 = pd.merge(B_relevant, C_relevant, on=["hhcode", 'msno'], how='inner', indicator=False)

# Evaluate merge 
#years04_05._merge.value_counts()

In [56]:
# Select people in school enrollment age (I would say 4-15 years old)

years04_05 = years04_05.loc[(years04_05["age"] > 4)&(years04_05["age"] <= 15)]
years04_05['year']=2004

In [57]:
# Change dtype to string (hhcode)
years04_05["hhcode"] = years04_05["hhcode"].astype("str")

In [58]:
#years04_05['subprovince']=years04_05['hhcode'].str.extract(r'(^w{3})')
years04_05['subprovince code']= years04_05['hhcode'].apply(lambda x: x[0:4])

In [59]:
years04_05["subprovince code"] = years04_05["subprovince code"].astype("int")

In [60]:
# Now we import the stratum name - code data
strat_name = pd.read_csv(
    "/Users/clarissaache/Documents/IDS 701/stratum0405_v2.csv"
)
strat_name.sample(5)

,province,subprovince code,subprovince
96,Balochistan,4232,Nasirabad
21,Punjab,1212,Sheikhupura
93,Balochistan,4202,Barkhan
38,Sindh,2042,Nowshero Freoze
35,Sindh,2012,Khairpur


In [61]:
# MERGE
years04_05_ = pd.merge(years04_05, strat_name, on=["subprovince code"], how='inner', indicator=True)

# Evaluate merge 
years04_05_._merge.value_counts()

both          25457
left_only         0
right_only        0
Name: _merge, dtype: int64

In [62]:
years04_05_.sample(5)

,hhcode,msno,age,sex,marital_status,ever_admitted,max_level_acheived,currently_enrrolled,why_not,region,province_x,year,subprovince code,province_y,subprovince,_merge
16264,3052002010.0,3,7,male,1:never married,no,NaN,NaN,1 too old / too young,rural,nwfp,2004,3052,NWFP,Shangla,both
13818,2122000407.0,5,8,female,1:never married,no,NaN,NaN,7 education not useful,rural,sindh,2004,2122,Sindh,Thatta,both
3286,1102002305.0,5,12,male,1:never married,yes,cl;ass 8,yes,NaN,rural,punjab,2004,1102,Punjab,Jhang,both
12556,2082003307.0,5,6,male,1:never married,no,NaN,NaN,3 too expansive,rural,sindh,2004,2082,Sindh,Larkana,both
13667,2112002305.0,6,15,male,1:never married,yes,class 9,yes,NaN,rural,sindh,2004,2112,Sindh,Badin,both


In [63]:
years04_05_.to_csv('/Users/clarissaache/Documents/IDS 701/uds-2022-ids-701-team-3/10_data_cleaning/clean_clarissa/years04_05_.csv', index_label="id")